In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
import string
import regex as re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import math
from copy import deepcopy
import json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
old_text = """The world is changing rapidly, and with it, our ways of doing things. Technology is advancing at a pace that is difficult to keep up with, and our lives are becoming more and more connected. We can communicate with people on the other side of the world in an instant, and access vast amounts of information at our fingertips.

As we embrace these changes, we also need to be mindful of their impact. Social media, for example, can be a powerful tool for connecting with others and sharing information. But it can also be a source of misinformation and negativity. We need to be critical of the information we consume and take steps to ensure that we are not perpetuating harmful narratives.

Ultimately, the key to navigating this rapidly changing world is adaptability. We need to be open to new ideas and ways of doing things, and be willing to learn and grow. By embracing change and staying curious, we can thrive in this new era of technology and interconnectedness."""

new_text = """The pace of change in today's world is nothing short of remarkable. Technological advancements are being made at an unprecedented rate, and our lives are becoming more and more intertwined with technology. From social media to artificial intelligence, these developments are transforming the way we live, work, and interact with one another.

However, it is important to acknowledge that not all of these changes are positive. Social media, for example, can be a powerful tool for connecting people and spreading information. But it can also be a breeding ground for negativity and hate speech. As we move forward, we need to be mindful of the impact these technologies can have on society, and take steps to address the negative effects.

At the heart of this changing landscape is the need for adaptability. In order to succeed in today's world, we need to be open to new ideas and be willing to learn and grow. By staying curious and embracing change, we can not only keep up with the rapid pace of technological change, but also use it to create a better future for ourselves and those around us."""

diff_text_one = """Meditation is an ancient practice that has been used for thousands of years to promote mental and physical wellbeing. In recent years, it has become increasingly popular in Western cultures, with numerous studies indicating the many benefits of regular meditation practice.

One of the main benefits of meditation is that it can reduce stress and anxiety. When we meditate, we focus our attention on the present moment, which helps to calm our minds and reduce feelings of worry and fear. Additionally, meditation can improve our ability to regulate our emotions, which can help us to cope better with difficult situations.

Another benefit of meditation is that it can improve our overall sense of wellbeing. Research has shown that regular meditation practice can lead to increased feelings of happiness, compassion, and gratitude. Furthermore, it can enhance our relationships with others, as we become more attuned to the emotions and needs of those around us."""

diff_text_two = """Plastic pollution is a growing global problem that is having a devastating impact on our planet's ecosystems. Plastic waste is polluting our oceans, harming wildlife, and even affecting human health. It is estimated that by 2050, there will be more plastic in the ocean than fish.

One of the dangers of plastic pollution is its impact on marine life. Plastic debris can be mistaken for food by marine animals, leading to ingestion and entanglement. This can result in injury, suffocation, and even death. Furthermore, plastic pollution can disrupt marine ecosystems, altering the balance of predator and prey species.

Another danger of plastic pollution is its impact on human health. Microplastics, which are small plastic particles, are now found in drinking water, seafood, and even the air we breathe. These particles can accumulate in our bodies over time, potentially leading to health problems such as inflammation and cancer. Furthermore, plastic pollution can have economic consequences, as it can damage tourism and fishing industries that rely on clean oceans and beaches."""

same_text_one = """Meditation is an ancient practice that has been used for thousands of years to promote mental and physical wellbeing. In recent years, it has become increasingly popular in Western cultures, with numerous studies indicating the many benefits of regular meditation practice.

One of the main benefits of meditation is that it can reduce stress and anxiety. When we meditate, we focus our attention on the present moment, which helps to calm our minds and reduce feelings of worry and fear. Additionally, meditation can improve our ability to regulate our emotions, which can help us to cope better with difficult situations.

Another benefit of meditation is that it can improve our overall sense of wellbeing. Research has shown that regular meditation practice can lead to increased feelings of happiness, compassion, and gratitude. Furthermore, it can enhance our relationships with others, as we become more attuned to the emotions and needs of those around us."""

same_text_two = """Meditation is an ancient practice that has been used for thousands of years to promote mental and physical wellbeing. In recent years, it has become increasingly popular in Western cultures, with numerous studies indicating the many benefits of regular meditation practice.

One of the main benefits of meditation is that it can reduce stress and anxiety. When we meditate, we focus our attention on the present moment, which helps to calm our minds and reduce feelings of worry and fear. Additionally, meditation can improve our ability to regulate our emotions, which can help us to cope better with difficult situations.

Another benefit of meditation is that it can improve our overall sense of wellbeing. Research has shown that regular meditation practice can lead to increased feelings of happiness, compassion, and gratitude. Furthermore, it can enhance our relationships with others, as we become more attuned to the emotions and needs of those around us."""

challenging_old_sent = 'You should not wear gloves while working.'
challenging_new_sent = 'You must wear gloves while working.'

sent_num_old = 'The length should be 45mm'
sent_num_new = 'The length should be 50mm'

sent_whole_old = 'There should be 9 factories and all of them should follow the guidelines'
sent_whole_new = 'There should be 10 factories and all of them should follow the guidelines'

document_dict = {old_text: new_text, diff_text_two:diff_text_one, same_text_one: same_text_two, challenging_old_sent:challenging_new_sent, sent_num_old: sent_num_new, sent_whole_old: sent_whole_new}

In [ ]:
def tokenize_paragraph(text):
  para_split = text.split("\n\n")
  para_dict = {}
  for para_idx in range(1, len(para_split)+1):
    para_dict[para_idx] = para_split[para_idx-1]
  # print(para_dict)
  return para_dict

In [ ]:
def tokenize_sentence(para_dict, sent_dict = None):
  if sent_dict == None:
    sent_dict = {}
  for k, v in para_dict.items():
    sent_tokenized = sent_tokenize(v)
    for sent_idx in range(1, len(sent_tokenized)+1):
      sent_dict[(k, sent_idx)] = sent_tokenized[sent_idx-1]
  # print(sent_dict)
  return sent_dict


In [ ]:
stop_words = stopwords.words('english')
stop_words.remove('not')
stop_words.extend(['.', 'a', 'an', 'the'])
stop_words.extend(string.punctuation)
stop_words = set(stop_words)
stemmer = PorterStemmer()

In [ ]:
def word_stemming(sent_dict):
  stemmed_sent = {}
  for k, v in sent_dict.items():
    word_tokenized = word_tokenize(v.lower())
    word_list = []
    word_count = {}
    for word_idx in range(len(word_tokenized)):
      if word_tokenized[word_idx] not in stop_words:
        word = stemmer.stem(word_tokenized[word_idx])
        word_list.append(word)
        word_count[word] = word_count.get(word, 0) + 1
    stemmed_sent[(k,v)] = [' '.join(word_list), word_count]
  # print(stemmed_sent)
  return stemmed_sent

In [ ]:
def process_text(text):
  para_dict = tokenize_paragraph(text)
  sent_dict = word_stemming(tokenize_sentence(para_dict))
  return sent_dict

In [ ]:
def term_frequency(old_sent_count_dict, new_sent_count_dict):
  old_sent_total_words = sum(old_sent_count_dict.values())
  new_sent_total_words = sum(new_sent_count_dict.values())
  for k, v in old_sent_count_dict.items():
    if k not in new_sent_count_dict.keys():
      new_sent_count_dict[k] = 0
    old_sent_count_dict[k] = (v/old_sent_total_words)
  for k, v in new_sent_count_dict.items():
    if k not in old_sent_count_dict.keys():
      old_sent_count_dict[k] = 0
    new_sent_count_dict[k] = (v/new_sent_total_words)
  return [dict(sorted(old_sent_count_dict.items())), dict(sorted(new_sent_count_dict.items()))]

In [ ]:
def match_similar(old_sent_dict, new_sent_dict):
  similarity_dict = {}
  for old_sent_key, old_sent in old_sent_dict.items():
    similarity_dict[(old_sent_key, old_sent[0])] = [None, -1]
    for new_sent_key, new_sent in new_sent_dict.items():
      old_sent_count, new_sent_count = term_frequency(deepcopy(old_sent[1]), deepcopy(new_sent[1]))
      old_sent_count_values = np.array(list(old_sent_count.values())).reshape(1, -1)
      new_sent_count_values = np.array(list(new_sent_count.values())).reshape(1, -1)
      cos_sim = cosine_similarity(old_sent_count_values, new_sent_count_values)
      #print(old_sent_key, new_sent_key, cos_sim)
      if similarity_dict[(old_sent_key, old_sent[0])][1] < float(cos_sim):
        similarity_dict[(old_sent_key, old_sent[0])] = [(new_sent_key, new_sent[0]), float(cos_sim)]
  return similarity_dict

In [ ]:
def detect_negation(sent_one, sent_two):
  neg_words = ['not', 'never', "n't", 'no', 'avoid']
  pattern = r'\b(?:{})\b'.format('|'.join(neg_words))
  if (bool(re.search(pattern, sent_one)) and not bool(re.search(pattern, sent_two))) or (bool(re.search(pattern, sent_two)) and not bool(re.search(pattern, sent_one))):
    return True

In [ ]:
def detect_number(sentence):
  sentence = sentence.replace(',', '')
  pattern = re.compile('[\d(\.\d)?]+')
  return re.findall(pattern, sentence)

In [ ]:
def match_filtering(similarity_dict):
  # print(similarity_dict)
  upper_threshold = 0.5
  lower_threshold = 0.2
  good_match = {}
  neg_sent = {}
  slight_match = {}
  no_match_old = []
  no_match_new = []
  for k, v in similarity_dict.items():
    if detect_negation(k[1], v[0][1]):
      neg_sent[k[0]] = v[0][0]
    elif v[1] >= upper_threshold:
      num_sent_old = detect_number(k[0][1])
      num_sent_new = detect_number(v[0][0][1])
      print(num_sent_old, num_sent_new)
      good_match[k[0]] = v[0][0]
    elif v[1] >= lower_threshold:
      slight_match[k[0]] = v[0][0]
    else:
      if v[0][0] not in no_match_old:
        no_match_old.append(k[0])
      if v[0][0] not in no_match_new:
        no_match_new.append(v[0][0])
  return [good_match, neg_sent, slight_match, no_match_old, no_match_new]

In [ ]:
def create_final_dictionary(similarity_ds):
  # print(similarity_ds)
  d = {'differences': {}, 'added': [], 'removed': []}
  for i in range(len(similarity_ds)):
    if i==1 or i==2:
      for k, v in similarity_ds[i].items():
        d['differences'][k] = v
    if i==3:
      for j in similarity_ds[i]:
        d['removed'].append(j)
    if i==4:
      for j in similarity_ds[i]:
        d['added'].append(j)
  return d

In [ ]:
def document_diff(old_document, new_document):
  old_sent_dict = process_text(old_document)
  new_sent_dict = process_text(new_document)
  similarity_dict = match_similar(old_sent_dict, new_sent_dict)
  similarity_ds = match_filtering(similarity_dict)
  return create_final_dictionary(similarity_ds)

In [ ]:
for k, v in document_dict.items():
  diff = document_diff(k, v)
  print(diff)

['.'] ['.']
['.'] ['.']
['.'] ['.']
{'differences': {((2, 1), 'As we embrace these changes, we also need to be mindful of their impact.'): ((3, 3), 'By staying curious and embracing change, we can not only keep up with the rapid pace of technological change, but also use it to create a better future for ourselves and those around us.'), ((2, 4), 'We need to be critical of the information we consume and take steps to ensure that we are not perpetuating harmful narratives.'): ((2, 4), 'As we move forward, we need to be mindful of the impact these technologies can have on society, and take steps to address the negative effects.'), ((3, 3), 'By embracing change and staying curious, we can thrive in this new era of technology and interconnectedness.'): ((3, 3), 'By staying curious and embracing change, we can not only keep up with the rapid pace of technological change, but also use it to create a better future for ourselves and those around us.'), ((1, 1), 'The world is changing rapidly, a

In [ ]:
with open('/content/drive/MyDrive/output.json', 'w') as f:
  json.dump(document_diff(diff_text_one, diff_text_two), f, indent=6)

In [ ]:
document_diff(sent_whole_old, sent_whole_new)

['9'] ['10']


{'differences': {}, 'added': [], 'removed': []}